<a href="https://colab.research.google.com/github/mtzig/NLP_CTF/blob/main/PyTorchDemo/Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A Demonstration of Training and Evaluating A Toxicity Classifier
Thomas Zeng
9/20/22

## Colab setup

This section is only pertinent if the notebook is run in Colab and not on a local machine.

If you're using colab, make sure to run below code to clone the repo

In [1]:
!git clone https://github.com/mtzig/NLP_CTF.git
%cd /content/NLP_CTF/PyTorchDemo

fatal: destination path 'NLP_CTF' already exists and is not an empty directory.
/content/NLP_CTF/PyTorchDemo


If you are on colab and using a gpu instance, below command will show the GPU google allocated to this session.

In [2]:
!nvidia-smi

Wed Sep 21 04:14:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Colab does not have the Python library `transformers` (which I use in below code) automatically installed, so we meed to manually install when we start up instance.

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Notebook Setup

Autoreload extension is used so that everyime you modify python files imported by notebook, you don't have to restart the notebook

In [4]:
%load_ext autoreload
%autoreload 2 # 2 is mode where every file you import gets reloaded when you run a block

In [5]:
import torch
from utils import load_jigsaw
from dataloaders import InfiniteDataLoader
from models import BertClassifier
from train_eval import train, evaluate

Pytorch can do it's computation in three modes.

1. Cuda: if you have a Nvidia gpu and you have a version of PyTorch installed that has cuda enabled (fastest speed)
2. Metal: if you have an Apple device with Apple Silicon (M1 or M2) -- this implementation of PyTorch is new, not feature complete and rather buggy (medium speed)
3. CPU: this is just the cpu (slowest speed)

In [6]:
if torch.cuda.is_available():
    print('Using GPU')
    DEVICE = torch.device('cuda')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    # macbooks can use metal if the right version of pytorch is installed
    print('Using Metal')
    DEVICE = torch.device('mps')
else:
    print('Using cpu')
    DEVICE = torch.device('cpu')

Using GPU


## Data Initialization

Pytorch requires its datasets to be ascessible following the [datasets api](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files).

Below I wrote a simple function to load in the [Jigsaw Dataset](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) that the paper [Counterfactual Fairness in
Text Classification through Robustness](https://dl.acm.org/doi/pdf/10.1145/3306618.3317950) used to train its toxicity classifier.

I use only a very small subset of the available data here for demonstration purposes. Specificaly 256 comments (128 toxic and 128 nontoxic) sampled randomly for the train set and test set respectively.

In [17]:
train_data, test_data = load_jigsaw(device=DEVICE, demo_mode=True) #demo_mode=True only loads a subset of entire dataset so as to make training faster for demonstration purposes

100%|██████████| 256/256 [00:00<00:00, 650.34it/s]


In Colab and Jupyter notebook, you can view the source of a function using `??`. Thus the we can look at the specific implementation of `load_jigsaw` function written above.

In [29]:
??load_jigsaw

PyTorch models receive data for training and inference through a dataloader. A dataloader samples from a dataset and returns a batch of samples each time it is called.

In [23]:
train_loader =  InfiniteDataLoader(train_data, batch_size=16)
test_loader = InfiniteDataLoader(test_data, batch_size=16)

## Model and Training Stuff Initialization

For our model, we use transfer learning, i.e. we use a pre-trained model -- DistilBert -- implemented by HuggingFace. Our `BertClassifier` is a simple wrapper arround their model.

In [24]:
model = BertClassifier(device=DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

An epoch is the number of times you go through your datase during training. That is you have trained for 1 epoch when you have seen every sample in your training dataset once.<br>
The loss function is the training objective we want our model to minimize.<br>
The optimizer is used at every time step i.e. everyime we compute the loss and its gradient. It is used to update the model weights.

In [25]:
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)

## Train and Evaluation

For traing, we train for 10 epochs. <br>
In general, you should (or more specifically are required to) train and evaluate using different datasets.

In [26]:
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train(train_loader, model, loss_fn, optimizer, verbose=True, use_tqdm=True)

Epoch 1/10


100%|██████████| 16/16 [00:06<00:00,  2.42it/s]


Average training loss: 0.684564895927906
Epoch 2/10


100%|██████████| 16/16 [00:06<00:00,  2.42it/s]


Average training loss: 0.5902475323528051
Epoch 3/10


...

Average training loss: 0.023043803055770695
Epoch 9/10


100%|██████████| 16/16 [00:06<00:00,  2.36it/s]


Average training loss: 0.016971627483144403
Epoch 10/10


100%|██████████| 16/16 [00:06<00:00,  2.35it/s]

Average training loss: 0.013704975601285696


We first evaluate our results on our train data

In [27]:
_ = evaluate(train_loader, model, get_loss=True, verbose=True)

100%|██████████| 16/16 [00:02<00:00,  5.98it/s]

Loss: 0.009806430898606777
Accuracy: 1.0, Sensitivity: 1.0, Specificity: 1.0


Our results are pretty amazing at first glance. However this is misleading as we are evaluating on the data we trained the model on. The high accuracies is a clear indication that the model is overfitting. An overfitted model is bad in general -- however in this case it is a clear indication that our code is working as expected.

If we then evaluate our reults on the test data:

In [28]:
_ = evaluate(test_loader, model, get_loss=True, verbose=True)

100%|██████████| 16/16 [00:02<00:00,  6.00it/s]

Loss: 0.5528503656387329
Accuracy: 0.80859375, Sensitivity: 0.8203125, Specificity: 0.796875


Clearly our results are much worse than before. Although they surprising are not that bad considering that we trained on only 256 examples (128 toxic, 128 non-toxic).

Although, these metrics do not take into account fairness -- so the model may very well be performing well due to some spurious correlation. Hence our comps project.
